<a href="https://colab.research.google.com/github/TVasantha02/Video_Offensive_words_Checker/blob/main/Video_Offensive_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Packages**

In [ ]:
!pip install openai-whisper
!pip install -U openai-whisper
!pip install git+https://github.com/openai/whisper.git
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git
!pip install ffmpeg
!pip install setuptools-rust
!pip install pydub
!pip install better_profanity
!pip install streamlit
!pip install SpeechRecognition

**Extracting audio from video|Transcribing audio to text|Checking for offensive words**

In [ ]:
%%writefile app.py
import os
import streamlit as st
from pydub import AudioSegment
from pydub.utils import mediainfo
import speech_recognition as sr
from better_profanity import profanity

def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()

    # Load the audio file
    audio = AudioSegment.from_file(audio_path)

    # Save audio as WAV (required by SpeechRecognition)
    wav_path = "extracted_audio.wav"
    audio.export(wav_path, format="wav")

    # Recognize speech using Google Web Speech API
    with sr.AudioFile(wav_path) as source:
        audio_data = recognizer.record(source)

    try:
        transcribed_text = recognizer.recognize_google(audio_data)
        return transcribed_text
    except sr.UnknownValueError:
        return None
    except sr.RequestError as e:
        return {"error": f"Speech recognition request failed; {e}"}

def check_video_offensiveness(video_file):
    # Save the uploaded file to disk
    with open("uploaded_video.mp4", "wb") as f:
        f.write(video_file.read())

    # Load the video file
    video_path = "uploaded_video.mp4"
    video_info = mediainfo(video_path)
    video_duration = float(video_info['duration']) * 1000  # in milliseconds

    # Extract audio
    audio_path = "extracted_audio.wav"
    audio = AudioSegment.from_file(video_path)
    audio.export(audio_path, format="wav")  # Export audio to WAV format

    # Perform transcription
    transcribed_text = transcribe_audio(audio_path)

    # Display the audio player
    st.markdown(f'**Audio Extraction**')
    audio_bytes = open(audio_path, "rb").read()  # Read WAV file as bytes
    st.audio(audio_bytes, format="audio/wav")

    # Display the transcription in an expander
    st.markdown('**Audio to Text Conversion**')

    if transcribed_text:
        # Display the extracted text inside brackets
        st.write(f"[{transcribed_text}]")

        # Check for offensive language
        st.markdown('**Result**')
        if contains_offensive(transcribed_text):
            st.error("Video contains offensive content.")
        else:
            st.success("Video is not offensive.")
    else:
        st.warning("Text extraction failed.")

    # Clean up
    os.remove(video_path)
    os.remove(audio_path)

def contains_offensive(text):
    # Set the profanity load_censor_words parameter to True
    profanity.load_censor_words()

    # Check if the text contains offensive language
    return profanity.contains_profanity(text)

def main():
    st.title("Video Offensiveness Checker")

    # File uploader for the video
    video_file = st.file_uploader("Upload Video", type=["mp4"])

    if video_file is not None:
        # Check offensiveness
        if st.button("Predict"):
            check_video_offensiveness(video_file)

if __name__ == "__main__":
    main()


In [ ]:
!curl icanhazip.com

In [ ]:
!streamlit run app.py --server.headless true --server.fileWatcherType none --browser.gatherUsageStats false &>/content/logs.txt &

In [ ]:
!npm install localtunnel

In [ ]:
!npx localtunnel --port 8501